# Connection Test

This is the connection and account test notebook for the course: 
* https://ufind.univie.ac.at/en/course.html?lv=052011&semester=2023W
* https://ufind.univie.ac.at/en/course.html?lv=052011&semester=2024W

## Basic interaction with `geth`

The most common client lib to interface with ethereum execution clients (`geth`,`ganache`,...) is **web3**.
* [web3.js](https://github.com/ethereum/web3.js/)
    - node.js version
    - [API](https://web3js.readthedocs.io/)
* [web3py](https://github.com/ethereum/web3.py)
    - python version
    - as I prefer python, we will use this one ;)
    - [API](https://web3py.readthedocs.io/)
    
For the tutorial we will only interact with `geth` which uses his own local development testnet blockchain. 
* [geth console](https://geth.ethereum.org/docs/interface/javascript-console) has a link to the supported API via the geth console, which should be the "full" `web3.js` API.
* [geth cli options](https://geth.ethereum.org/docs/interface/command-line-options) only needed when running geth from the command line

## Connect to your local geth node (bob)

Note that in our case `geth` is running in a private PoA chain setup.
Therefore, some configuration parameters are different (see https://web3py.readthedocs.io/en/stable/middleware.html#geth-style-proof-of-authority for more details). 

In [1]:
import web3
#from web3.middleware import geth_poa_middleware
from web3.middleware import ExtraDataToPOAMiddleware


w3 = web3.Web3(web3.Web3.HTTPProvider("http://geth-client-cnt:8545"))
# check if connection was successful
assert w3.is_connected()

#w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.middleware_onion.inject(ExtraDataToPOAMiddleware, layer=0)

In [2]:
w3.geth.admin.node_info()

AttributeDict({'id': '934d8756c78b549c9e6b082047749387c76cc03266de0fb079b9ace14e5b9edf',
 'name': 'Geth/bob/v1.10.3-stable-991384a7/linux-amd64/go1.21.5',
 'enode': 'enode://82affa0044f164615645f22d3ff71063ade23aedc87b4ff27fde8c4a395f2da4566277119c3b1bfb6947aa4396304ca11424d8ac731bad78d9f219d11943da87@127.0.0.1:30303?discport=0',
 'enr': 'enr:-Ja4QDlwgUHTGOPzxoKtxav-kNe3j7ARS-WrIbZtwfsRf2C5AQpiilvcVQ-nZiilKunDctYV4kx5RIVAGdv871MTNbEBg2V0aMfGhLpYo26AgmlkgnY0gmlwhH8AAAGJc2VjcDI1NmsxoQOCr_oARPFkYVZF8i0_9xBjreI67ch7T_J_3oxKOV8tpIRzbmFwwIN0Y3CCdl8',
 'ip': '127.0.0.1',
 'ports': AttributeDict({'discovery': 0, 'listener': 30303}),
 'listenAddr': '[::]:30303',
 'protocols': AttributeDict({'eth': AttributeDict({'network': 20250101,
   'difficulty': 1,
   'genesis': '0x0ddaabe3a62313c0bb1b3d7f2da96d1752c4ab70d8b18c4e4914c1a439cc2d73',
   'config': AttributeDict({'chainId': 20250101,
    'homesteadBlock': 0,
    'eip150Block': 0,
    'eip150Hash': '0x000000000000000000000000000000000000000000000

In [3]:
# display the client version of the node we are connected to
w3.client_version

'Geth/bob/v1.10.3-stable-991384a7/linux-amd64/go1.21.5'

In [4]:
# display network ID of client you are connected to
w3.net.version

'20250101'

In [5]:
w3.eth.chain_id

20250101

In [6]:
# Display number of connected peers should be 1
w3.net.peer_count

0

In [7]:
# check if node is "up-to-date" with blockchain, 
# i.e., not syncing anymore
w3.eth.syncing

False

In [8]:
#w3.eth.mining
w3.manager.request_blocking("eth_mining", []) # call directly geth API
#w3.eth.is_mining # not yet implemented in this version of web3py

False

## Connect to our geth node (alice) with your geth node (bob)

To make this work the wireshark VPN for the secenv needs to be working on your host as usually.
In addition you need the `enode` of our geth PoA node. 

To connect to a geth node a `enode` ulr is required. This contains a public key of this node as well as its domain name (or IP) and prot. 

*Only informational*: To manually extract the enode from a running geth node (e.g., our server), cat the `nodekey` file which contains the private key and use the `bootnode` tool to generate the associated public key, i.e., enode. (see https://ethereum.stackexchange.com/questions/53086/go-ethereum-get-the-enode-before-starting-geth)

```bash
$ cat $DATADIR/geth/nodekey
$ bootnode -nodekeyhex $(cat $DATADIR/geth/nodekey) -writeaddress
```

Alternatively, run the following command on the host (Container) where `geth` is running:
```bash
$ geth attach http://127.0.0.1:8545
Welcome to the Geth JavaScript console!

instance: Geth/alice/v1.10.3-stable-991384a7/linux-amd64/go1.21.5
coinbase: 0xf78342e0cdb2188e3b5c9663b1c6d6d5af68bc97
at block: 616 (Fri Dec 20 2024 21:13:58 GMT+0100 (CET))
 datadir: /app/datadir/alice
 modules: admin:1.0 clique:1.0 debug:1.0 eth:1.0 miner:1.0 net:1.0 personal:1.0 rpc:1.0 txpool:1.0 web3:1.0

To exit, press ctrl-d
> admin.nodeInfo
{
  enode: "enode://fd90f39e40633934f1613ab08bd4917d3aa28abba28cc4748b544650cfffe1ab40890f4a9d85ca9c0cf28845633291ca06a5716b86f5232b32374b82e2e3cff6@127.0.0.1:30303?discport=0",
  ...
```

The enode of our server is: 

In [9]:
!pwd

/smartenv


In [10]:
!cat geth/enodes

enode://fd90f39e40633934f1613ab08bd4917d3aa28abba28cc4748b544650cfffe1ab40890f4a9d85ca9c0cf28845633291ca06a5716b86f5232b32374b82e2e3cff6@eth-smart.secenv:30303?discport=0


In [11]:
enodes_file = !cat geth/enodes
enode = enodes_file[0]

In [12]:
assert enode == "enode://fd90f39e40633934f1613ab08bd4917d3aa28abba28cc4748b544650cfffe1ab40890f4a9d85ca9c0cf28845633291ca06a5716b86f5232b32374b82e2e3cff6@eth-smart.secenv:30303?discport=0"

Or replace domain name with IP if DNS & wireguard makes problems:

In [13]:
!ping -c 1 eth-smart.secenv

PING eth-smart.secenv (10.81.0.38) 56(84) bytes of data.
64 bytes from 10.81.0.38 (10.81.0.38): icmp_seq=1 ttl=62 time=2.03 ms

--- eth-smart.secenv ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 2.030/2.030/2.030/0.000 ms


In [14]:
#enode = "enode://fd90f39e40633934f1613ab08bd4917d3aa28abba28cc4748b544650cfffe1ab40890f4a9d85ca9c0cf28845633291ca06a5716b86f5232b32374b82e2e3cff6@10.81.0.38:30303?discport=0"

In [15]:
w3.net.peer_count # peer count before connecting, should be zero

0

In [16]:
w3.geth.admin.add_peer(enode)

True

In [17]:
import time
time.sleep(4)

In [18]:
w3.net.peer_count # peer count after connecting, should be one

1

In [19]:
# You should be connected to our node now
assert w3.net.peer_count >= 1,"Not connected to our node!"

In [20]:
# get the information about your peer(s)
peers = w3.geth.admin.peers()
peers

[AttributeDict({'enode': 'enode://fd90f39e40633934f1613ab08bd4917d3aa28abba28cc4748b544650cfffe1ab40890f4a9d85ca9c0cf28845633291ca06a5716b86f5232b32374b82e2e3cff6@10.81.0.38:30303?discport=0',
  'id': '2af0d687348cffa0a8d8e827abd4e1e5f2c68e4beca2dfaa768856a9dbba522e',
  'name': 'Geth/alice/v1.10.3-stable-991384a7/linux-amd64/go1.21.5',
  'caps': ['eth/65', 'eth/66', 'snap/1'],
  'network': AttributeDict({'localAddress': '172.21.0.4:43174',
   'remoteAddress': '10.81.0.38:30303',
   'inbound': False,
   'trusted': False,
   'static': True}),
  'protocols': AttributeDict({'eth': AttributeDict({'version': 66,
    'difficulty': 167,
    'head': '0x89dd008c637851dd03a8a4756f24d2c16417254bb3a9284fc50c7bab9b0b5879'}),
   'snap': AttributeDict({'version': 1})})})]

In [21]:
assert "alice" in peers[0]["name"]

At the beginning your node might sync a while till it has reached the tip of the chain.
If `w3.eth.syncing` return `False` it is up-to-date

In [22]:
w3.eth.syncing

False

## Check state of blockchain 

In [23]:
# return current blockchain head of node
w3.eth.block_number

83

In [24]:
# should be greater
!sleep 13
w3.eth.block_number

85

In [25]:
# block 0 ist the genesis block of this testnet blockchain
genesis_block = w3.eth.get_block(0)
genesis_block

AttributeDict({'difficulty': 1,
 'proofOfAuthorityData': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000f78342e0cdb2188e3b5c9663b1c6d6d5af68bc970000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'gasLimit': 15000000,
 'gasUsed': 0,
 'hash': HexBytes('0x0ddaabe3a62313c0bb1b3d7f2da96d1752c4ab70d8b18c4e4914c1a439cc2d73'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'miner': '0x00000000000

As you see, the genesis block is special as it is the only block with no parent hash:

In [26]:
genesis_block['number']

0

In [27]:
genesis_block['parentHash']

HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000')

The [genesis.json](../geth/genesis.json) file shows the node(s) that are allowed to mine blocks and the accounts that have been seeded with coins. 
The client we are currently connected to is not part of the PoA nodes that are allowed to create new blocks:

In [28]:
#assert genesis_block['hash'].hex() == '6bec1279ca6efde179db76897011152e3b1d4c89f11298b501a501511fa016b8'

In [28]:
assert genesis_block['hash'].hex() == '0ddaabe3a62313c0bb1b3d7f2da96d1752c4ab70d8b18c4e4914c1a439cc2d73'

## Import your personal key

Connect to `nc eth-smart.secenv 4242` and collect you personal private key for your account to interact with your personalized smart contracts. 

In [29]:
!printf '1\r\n' | nc -w 10 -q 10 eth-smart.secenv 4242 | head -n 9


Ethereum challenge helper
Hello, here you get your personal private key:
1. Option 1: Get Key
2. Option 2: Get Key json
3. Option 3: Get personal contract addressed
4. Option 4: Grade me/get Flag
5. Option 5: Exit
Your personal private key:


Initially there are no accounts if you havend stored your json key in the `keystore` folder yet:

In [31]:
!ls /keystore

In [32]:
w3.eth.accounts

[]

You can get your key from our terminal server application at `eth-smart.secenv` running at port `4242`

In [33]:
my_private_key = !printf "1\r\n" | nc -W 3 eth-smart.secenv 4242 | tail -n1

In [34]:
account = w3.eth.account.from_key(my_private_key[0])

In [35]:
account.address

'0x53B30788b6a47261be56a851C22B155cd3b84735'

By providing a password, you can gerate a keystore file from this account `account.encrypt("password")` which you can write to disk.

In [36]:
import util
util.write_keystore_file("/keystore/",account.encrypt("password"))

writing to UTC--2024-12-20T23-42-04.050655Z--53B30788b6a47261be56a851C22B155cd3b84735 ...


In [41]:
import time
time.sleep(4)

In [42]:
w3.eth.accounts

['0x53B30788b6a47261be56a851C22B155cd3b84735']

In [43]:
# you should set this account as default account so that it is used
# by all functions requireing access
w3.eth.default_account = w3.eth.accounts[0]
w3.eth.default_account 

'0x53B30788b6a47261be56a851C22B155cd3b84735'

To use/access an account, e.g., for making a transaction you need to unlock it first by providing its password and a time in seconds for how long this account should be unlocked (0 means without limit).

In [50]:
#w3.geth.personal.unlock_account(w3.eth.accounts[0],"password",0)